In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns
import warnings
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier


## Data Collection

In [ ]:
space_ship = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

## Data Wrangling

In [ ]:
Unknow ='Unknow'

def split(sep,index,x):
    if str(x) == str(np.nan):
        return Unknow
    else:
        return str(x).split(sep)[index]
def age_bean(x):
    if str(x) == str(np.nan):
        return Unknow
    elif (x<18):
        return '0-18'
    elif (x<40):
        return '18-40'
    else:
        return '40-'
def price_bean(x, limit):
    if (str(x) == str(np.nan)):
        return Unknow
    elif (float(x)< limit):
        return '<'+ str(limit)
    else :
        return '>'+str(limit)
def create_new_features(df):
    space_ship = df.copy()
    space_ship['GroupId'] = space_ship['PassengerId'].apply(lambda x: split('_',0,x))
    space_ship['Cabin_Deck'] = space_ship['Cabin'].apply(lambda x: split('/',0,x))
    space_ship['Cabin_Num'] = space_ship['Cabin'].apply(lambda x: split('/',1,x))
    space_ship['Cabin_Side'] = space_ship['Cabin'].apply(lambda x: split('/',2,x))
    space_ship['FirstName'] = space_ship['Name'].apply(lambda x: split(' ',0,x))
    space_ship['LastName'] = space_ship['Name'].apply(lambda x: split(' ',1,x))
    space_ship['Age_Bean'] = space_ship['Age'].apply(lambda x: age_bean(x))
    space_ship['RoomService_Bean'] = space_ship['RoomService'].apply(lambda x: price_bean(x,400))
    space_ship['ShoppingMall_Bean'] = space_ship['ShoppingMall'].apply(lambda x: price_bean(x,1000))
    space_ship['FoodCourt_Bean'] = space_ship['FoodCourt'].apply(lambda x: price_bean(x,1600))
    return space_ship
def replace_unknow(df):
    space_ship = df.copy()
    space_ship['HomePlanet'] = space_ship['HomePlanet'].replace(Unknow, 'Mars')
    space_ship['CryoSleep'] = space_ship['CryoSleep'].replace(Unknow, False)
    space_ship['Cabin_Deck'] = space_ship['Cabin_Deck'].replace(Unknow, 'A')
    space_ship['Cabin_Deck'] = space_ship['Cabin_Deck'].replace(Unknow, 'A')
    space_ship['Cabin_Side'] = space_ship['Cabin_Side'].replace(Unknow, 'S')
    space_ship['Cabin_Num'] = space_ship['Cabin_Num'].replace(Unknow, -1)
    space_ship['Cabin_Num'] = space_ship['Cabin_Num'].apply(lambda x: int(x))
    space_ship['Cabin_Num'] = space_ship['Cabin_Num'].replace(-1, space_ship['Cabin_Num'].mean())
    space_ship['Destination'] = space_ship['Destination'].replace(Unknow, 'PSO J318.5-22')
    space_ship['VIP'] = space_ship['VIP'].replace(Unknow, False)
    space_ship['Age_Bean'] = space_ship['Age_Bean'].replace(Unknow, '40-')
    space_ship['RoomService_Bean'] = space_ship['RoomService_Bean'].replace(Unknow, '<400')
    space_ship['ShoppingMall_Bean'] = space_ship['ShoppingMall_Bean'].replace(Unknow, '<1000')
    space_ship['FoodCourt_Bean'] = space_ship['FoodCourt_Bean'].replace(Unknow, '<1600')
    return space_ship
def replace_nan(df):
    numrical = [
        # 'Age',
        'RoomService',
        'FoodCourt',
        'ShoppingMall',
        'Spa',
        'VRDeck'
    ]
    space_ship = df.copy()
    space_ship['Age'].fillna(space_ship['Age'].mean(), inplace=True)
    for col in numrical:
        space_ship[col].fillna(random.randint(100, 2000), inplace=True)
   
    return space_ship
        

In [ ]:

space_ship = create_new_features(space_ship)
test = create_new_features(test)

In [ ]:
numerical = [
    'Age',
    'RoomService',
    'FoodCourt',
    'ShoppingMall',
    'Spa',
    'VRDeck'
]
categorical = [
        'PassengerId',
        'GroupId',
        'HomePlanet',
        'CryoSleep',
        'Cabin',
        'Cabin_Deck',
        'Cabin_Num',
        'Cabin_Side',
        'Destination',
        'VIP',
        'FirstName',
        'LastName',
        'Age_Bean',
        'RoomService_Bean',
        'ShoppingMall_Bean',
        'FoodCourt_Bean'
    ]
target ='Transported'
def unknow_categorical(df):
    space_ship = df.copy()
    for col in categorical:
        space_ship[col].fillna('Unknow', inplace=True)
    return space_ship

In [ ]:
test = unknow_categorical(test)
space_ship = unknow_categorical(space_ship)

## Data Visualization and Sumary

### Summarizing a population

In [ ]:
all_columns = numerical + categorical
all_columns

In [ ]:
df = space_ship[all_columns].copy()
for col in numerical:
    plt.subplots(figsize=(18,9))
    plt.subplot(1,2,1)
    df[col].plot.hist()
    plt.title(f'Histogram - {col.upper()}')
    plt.xlabel(col)
    plt.subplot(1,2,2)
    df[col].plot.box(vert=True)
    plt.title(f'BoxPlot - {col.upper()}')
    plt.ylabel(col)
    plt.show()

In [ ]:
# for col in categorical:
#     plt.subplots(figsize=(18,9))
#     df[col].value_counts().plot.barh()
#     # plt.tight_layout()
#     plt.title(col)
#     plt.show()

### Comparing populations

In [ ]:
# How Passenger has been transported dependent on each group of each features

df = space_ship[categorical +['Transported']].copy()
for col in categorical:
    possibilities = list(df[col].unique())
    plt.subplots(figsize=(20,4))
    for i,poss in enumerate(possibilities):
        BM = df[col] == poss
        plt.subplot(1,len(possibilities), i+1)
        df[BM]['Transported'].value_counts().plot.bar()
        # plt.tight_layout()
        plt.title(f'{col} - {poss}')
        plt.xlabel('Transported')
    plt.show()

In [ ]:
title = 'Age Distribution - Transported'
columns = [
    'Age',
    'Transported'
]
feature_study ='Age'
target ='Transported'
df = space_ship[columns].copy()
target_possibilities = df[target].unique()
plt.subplots(figsize=(15,8))
for poss in target_possibilities:
    BM = df[target] == poss
    plt.hist(df[BM][feature_study],label=poss, histtype='step')
plt.title(title)
plt.xlabel(feature_study)
plt.ylabel('Count')
plt.legend()
plt.show()

As we can see, we can  create tree beans for Age feature ( `0-18`, `18-40`,`40-`)
* Passenger in `0-18` was more Transported
* Passenger in `18-40` was more not Transported
This is because we create `Age_Bean`

In [ ]:
space_ship['Age_Bean'].unique()

In [ ]:
numerical = [
    # 'Age',
    'RoomService',
    # 'FoodCourt',
    # 'ShoppingMall',
    'Spa',
    'VRDeck'
]
for col in numerical:
    title = col + ' Distribution - Transported'
    columns = [
        col,
        'Transported'
    ]
    feature_study =col
    target ='Transported'
    df = space_ship[columns].copy()
    target_possibilities = df[target].unique()
    plt.subplots(figsize=(15,3))
    for poss in target_possibilities:
        BM = df[target] == poss
        plt.hist(df[BM][feature_study],label=poss, histtype='step')
    plt.title(title)
    plt.xlabel(feature_study)
    plt.ylabel('Count')
    plt.text(400,0,'limit')
    plt.legend()
    plt.show()

Passenger that spend more than:
*`400` in luxe like `RoomService`, `Spa` and `VRDeck` has high chance to be transported*

In [ ]:
numerical = [
    # 'Age',
    # 'RoomService',
    'FoodCourt',
    # 'ShoppingMall',
    # 'Spa',
    # 'VRDeck'
]
for col in numerical:
    title = col + ' Distribution - Transported'
    columns = [
        col,
        'Transported'
    ]
    feature_study =col
    target ='Transported'
    df = space_ship[columns].copy()
    target_possibilities = df[target].unique()
    plt.subplots(figsize=(15,3))
    for poss in target_possibilities:
        BM = df[target] == poss
        plt.hist(df[BM][feature_study],label=poss, histtype='step')
    plt.title(title)
    plt.xlabel(feature_study)
    plt.ylabel('Count')
    plt.text(1600,0,'limit')
    plt.legend()
    plt.show()


Passenger that spend  more than `1600` in `FoodCourt` has more chance to be not Transported

In [ ]:
numerical = [
    # 'Age',
    # 'RoomService',
    # 'FoodCourt',
    'ShoppingMall',
    # 'Spa',
    # 'VRDeck'
]
for col in numerical:
    title = col + ' Distribution - Transported'
    columns = [
        col,
        'Transported'
    ]
    feature_study =col
    target ='Transported'
    df = space_ship[columns].copy()
    target_possibilities = df[target].unique()
    plt.subplots(figsize=(15,3))
    for poss in target_possibilities:
        BM = df[target] == poss
        plt.hist(df[BM][feature_study],label=poss, histtype='step')
    plt.title(title)
    plt.xlabel(feature_study)
    plt.ylabel('Count')
    plt.text(1000,0,'limit')
    plt.legend()
    plt.show()

Passenger that spend  more than `1000` in `ShoppingMall` has more chance to be not Transported

In [ ]:
numerical = [
    # 'Age',
    # 'RoomService',
    # 'FoodCourt',
    # 'ShoppingMall',
    # 'Spa',
    'VRDeck'
]
for col in numerical:
    title = col + ' Distribution - Transported'
    columns = [
        col,
        'Transported'
    ]
    feature_study =col
    target ='Transported'
    df = space_ship[columns].copy()
    target_possibilities = df[target].unique()
    plt.subplots(figsize=(15,3))
    for poss in target_possibilities:
        BM = df[target] == poss
        plt.hist(df[BM][feature_study],label=poss, histtype='step')
    plt.title(title)
    plt.xlabel(feature_study)
    plt.ylabel('Count')
    plt.text(400,0,'limit')
    plt.legend()
    plt.show()

## Modeling

In [ ]:

numerical = [
        'Age',
        'RoomService',
        'FoodCourt',
        'ShoppingMall',
        'Spa',
        'VRDeck',
    ]
label = [
         'CryoSleep',
         'Cabin_Num',
         'Cabin_Deck',
         'Cabin_Side',
         'VIP',
        'GroupId',
]
categorical = [ 
          'Destination',
        'HomePlanet',
        # 'Age_Bean'
    ]
target = space_ship['Transported']

df = replace_nan(space_ship)
df = replace_unknow(df)

df_test = replace_nan(test)
df_test = replace_unknow(df_test)

df = df[numerical + categorical + label]
df_test = df_test[numerical + categorical + label]

In [ ]:
le = LabelEncoder()
for col in label:
    le.fit(df[col])
    df[col] = le.transform(df[col])
    le.fit(df_test[col])
    df_test[col] = le.transform(df_test[col])

In [ ]:
X = df.copy()
y = target
data_train, data_test, target_train, target_test = train_test_split(X, y, random_state=42)

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical),
    ('standard_scaler', numerical_preprocessor, numerical + label)
    ])

In [ ]:
warnings.filterwarnings('ignore')

models = [
          ('Logistic', LogisticRegression(max_iter=20000)),
          ('RandomForestClassifier', RandomForestClassifier()),
          ('XGBClassifier',XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=500, objective='binary:logistic', booster='gbtree')),
          ('LinearDiscriminantAnalysis', LinearDiscriminantAnalysis(n_components=1)),
          ('KNeighborsClassifier', KNeighborsClassifier(n_neighbors=5)),
          ('GaussianNB', GaussianNB()),
          # ('MultinomialNB', MultinomialNB()),
          ('DecisionTreeClassifier', DecisionTreeClassifier()),
          ('SVC', SVC())
]
best_model = None
best_score = 0
for name, model in models:
    model_pipeline = make_pipeline(preprocessor, model)
    model_pipeline.fit(data_train,target_train)
    y_predict = model_pipeline.predict(data_test)
    score = accuracy_score(y_predict, target_test)
    if(score > best_score):
        best_model = model_pipeline
        best_score = score
    print(name, score)

In [ ]:
X_test = df_test.copy()
# Submition
y_predict = best_model.predict(X_test)
submission = test[['PassengerId']]
submission['Transported'] = y_predict
submission.set_index("PassengerId")[["Transported"]].to_csv("submission.csv")

In [ ]:
!ls